### Not the best Housing Prices solution out there, but will give you a pretty decent score. :)

## Data Pre-processing

### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from scipy.stats import norm, skew
from scipy import stats
from scipy.special import boxcox1p
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

### Import the Data

In [ ]:
HousingPrices_train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv',sep = ",", header = 0)
HousingPrices_test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv',sep = ",", header = 0)
#Inject SalePrice in test
HousingPrices_test['SalePrice'] = 0
print("Train Shape: " + str(HousingPrices_train.shape))
print("Test Shape: " + str(HousingPrices_test.shape))

In [ ]:
HousingPrices_full = pd.concat([HousingPrices_train,HousingPrices_test])
HousingPrices_full.shape

##### Take a Peek at the Dataset

In [ ]:
HousingPrices_full.head(5)

In [ ]:
HousingPrices_full.tail(5)

In [ ]:
HousingPrices_full.describe()

In [ ]:
HousingPrices_full.replace('',np.nan,inplace=True)

##### Transform necessary features to 'object' type

In [ ]:
HousingPrices_full['MSSubClass'] = HousingPrices_full['MSSubClass'].astype('object')
HousingPrices_full['OverallCond'] = HousingPrices_full['OverallCond'].astype('object')
HousingPrices_full['YrSold'] = HousingPrices_full['YrSold'].astype('object')
HousingPrices_full['MoSold'] = HousingPrices_full['MoSold'].astype('object')

### Check for Missing Values

##### Visualize using Missingness Map

In [ ]:
sns.set_palette("GnBu_d")
plt.title("Missingess Map")
plt.rcParams['figure.figsize'] = (40, 40)
sns.heatmap(HousingPrices_full.isnull(), cbar=False)

In [ ]:
for i,j in HousingPrices_full.isnull().sum().iteritems():
    print(i,j)

### Impute for Missing Values

##### Lot Frontage - Fill with Median

In [ ]:
HousingPrices_full['LotFrontage'].fillna(value=
                                    HousingPrices_full['LotFrontage'].median(),
                                    inplace = True)

##### FireplaceQU - Drop Feature

In [ ]:
HousingPrices_full.drop('FireplaceQu', axis = 1, inplace = True)

##### GarageQual, GarageType, GarageFinish, GarageCond - Fill with None

In [ ]:
d = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']
for i in d:
    HousingPrices_full[i] = HousingPrices_full[i].fillna('None')

##### GarageArea, GarageYrBlt, GarageCars = Fill with 0s

In [ ]:
d = ['GarageArea', 'GarageYrBlt', 'GarageCars']
for i in d:
    HousingPrices_full[i] = HousingPrices_full[i].fillna(0)

##### BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath and BsmtHalfBath - Fill with 0s

In [ ]:
d = ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath']
for i in d:
    HousingPrices_full[i] = HousingPrices_full[i].fillna(0)

##### BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1 and BsmtFinType2 - Fill with None

In [ ]:
d = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
for i in d:
    HousingPrices_full[i] = HousingPrices_full[i].fillna('None')

##### PoolQC - Fill with None

In [ ]:
HousingPrices_full['PoolQC'].fillna("None",inplace = True)

##### MiscFeature - Fill with None

In [ ]:
HousingPrices_full['MiscFeature'].fillna("None",inplace = True)

##### Alley - Fill with None

In [ ]:
HousingPrices_full['Alley'].fillna("None",inplace = True)

##### Fence - Fill with None

In [ ]:
HousingPrices_full['Fence'].fillna("None",inplace = True)

##### MasVnrArea - Fill with 0

In [ ]:
HousingPrices_full["MasVnrArea"] = HousingPrices_full["MasVnrArea"].fillna(0)

 ##### MasVnrType - Fill with None

In [ ]:
HousingPrices_full["MasVnrType"] = HousingPrices_full["MasVnrType"].fillna("None")

##### MSZoning - Fill with the most frequent class

In [ ]:
HousingPrices_full['MSZoning'].fillna(value=
                                   HousingPrices_full['MSZoning'].value_counts().idxmax(),
                                   inplace = True)

##### Utilities - Drop Feature

In [ ]:
HousingPrices_full.drop('Utilities', axis = 1, inplace = True)

##### Functional - Fill with Typ

In [ ]:
HousingPrices_full['Functional'].fillna("Typ",inplace = True)

##### KitchenQual - Fill with most frequent class

In [ ]:
HousingPrices_full['KitchenQual'].fillna(value=
                                   HousingPrices_full['KitchenQual'].value_counts().idxmax(),
                                   inplace = True)

##### Electrical - Fill with the most frequent class

In [ ]:
HousingPrices_full['Electrical'].fillna(value=
                                   HousingPrices_full['Electrical'].value_counts().idxmax(),
                                   inplace = True)

##### Exterior1st - Fill with most frequent class

In [ ]:
HousingPrices_full['Exterior1st'].fillna(value=
                                   HousingPrices_full['Exterior1st'].value_counts().idxmax(),
                                   inplace = True)

##### Exterior2nd - Fill with most frequent class

In [ ]:
HousingPrices_full['Exterior2nd'].fillna(value=
                                   HousingPrices_full['Exterior2nd'].value_counts().idxmax(),
                                   inplace = True)

##### SaleType - Fill with most frequent class

In [ ]:
HousingPrices_full['SaleType'].fillna(value=
                                   HousingPrices_full['SaleType'].value_counts().idxmax(),
                                   inplace = True)

### Check for missing values again

In [ ]:
HousingPrices_full.info()

In [ ]:
plt.rcParams['figure.figsize'] = (20, 20)
sns.heatmap(HousingPrices_full.isnull(), cbar=False)

##### Split back to train and test

In [ ]:
HousingPrices_train = HousingPrices_full[0:1460]
HousingPrices_test = HousingPrices_full[1460:2919]

### Data Exploration

##### Check for skewness of features

In [ ]:
full_numeric = HousingPrices_full.dtypes[HousingPrices_full.dtypes != "object"].index
skewed_features = HousingPrices_full[full_numeric].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkewness of Numeric Features \n")
skewness = pd.DataFrame({'skew' :skewed_features})
skewness

##### Visualize some

In [ ]:
#Lot Frontage
plt.rcParams['figure.figsize'] = (7.0, 5.0) #Adjust values as necessary
sns.distplot(HousingPrices_full['LotFrontage'], fit=norm);
fig = plt.figure()
res = stats.probplot(HousingPrices_full['LotFrontage'], plot=plt)

In [ ]:
#LotArea
plt.rcParams['figure.figsize'] = (7.0, 5.0) #Adjust values as necessary
sns.distplot(HousingPrices_full['LotArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(HousingPrices_full['LotArea'], plot=plt)

##### Apply boxcox1p Transformation (to deal with features with 0 values) to all features with >0.7 absolute skewness

In [ ]:
to_transform = skewness[abs(skewness) > 0.7]
to_transform = to_transform[to_transform['skew'].notna()]
to_transform

In [ ]:
greater070 = list(to_transform.index)
greater070.remove('SalePrice')
for i in greater070:
    HousingPrices_full[i] = boxcox1p(HousingPrices_full[i], 0.15)

##### Check LotFrontage and Lot Area again

In [ ]:
#Lot Frontage
plt.rcParams['figure.figsize'] = (7.0, 5.0) #Adjust values as necessary
sns.distplot(HousingPrices_full['LotFrontage'], fit=norm);
fig = plt.figure()
res = stats.probplot(HousingPrices_full['LotFrontage'], plot=plt)

In [ ]:
plt.rcParams['figure.figsize'] = (7.0, 5.0) #Adjust values as necessary
sns.distplot(HousingPrices_full['LotArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(HousingPrices_full['LotArea'], plot=plt)

##### Dealing with the Dependent Variable - SalePrice

In [ ]:
plt.rcParams['figure.figsize'] = (7.0, 5.0) #Adjust values as necessary
sns.distplot(HousingPrices_train['SalePrice'], fit=norm);
fig = plt.figure()
res = stats.probplot(HousingPrices_train['SalePrice'], plot=plt)

##### Normal Log Transformation

In [ ]:
HousingPrices_train['SalePrice'] = np.log(HousingPrices_train['SalePrice'])
sns.distplot(HousingPrices_train['SalePrice'], fit=norm);
fig = plt.figure()
res = stats.probplot(HousingPrices_train['SalePrice'], plot=plt)

##### Look how much features are correlated by using the Correlation Plot

In [ ]:
plt.rcParams['figure.figsize'] = (15.0, 13.0) #Adjust values as necessary
plt.title("Correlation Plot")
sns.heatmap(HousingPrices_train.corr())

## Feature Engineering

### Additional

##### Trick: Combine BsmtFinSF1, BsmtFinSF2, 1stFlrSF and 2ndFlrSF

In [ ]:
HousingPrices_full['TotalSF'] = (
    HousingPrices_full['BsmtFinSF1'] +
    HousingPrices_full['BsmtFinSF2'] +
    HousingPrices_full['1stFlrSF'] +
    HousingPrices_full['2ndFlrSF']
)

### One-Hot Encoding using pd.getdummies

In [ ]:
todummify = list(HousingPrices_full.select_dtypes(include=['object']).columns)
HousingPrices_full = pd.get_dummies(HousingPrices_full, columns=todummify)
#Convert them first to category type so they wont get affected in VIF - elimination method
tocategorify = list(HousingPrices_full.select_dtypes(include=['uint8']).columns)
HousingPrices_full[tocategorify] = HousingPrices_full[tocategorify].astype('category')
HousingPrices_full.info(verbose=True)

##### Splitting Again

In [ ]:
HousingPrices_train = HousingPrices_full[0:1460]
HousingPrices_test = HousingPrices_full[1460:2919]

### Define IVs and DV (X & y)

In [ ]:
X = HousingPrices_train.drop(['SalePrice',
                              'Id'], axis = 1) 
y = HousingPrices_train['SalePrice']
print("Dependent Variables")
display(X.head())
print("Independent Variable")
display(y.to_frame().head())

## Feature Selection with Lasso

###### If the feature is irrelevant, lasso penalizes it’s coefficient and shrinks it. Hence the features with coefficient = 0 are removed and the rest are taken. Dropping one column from the one-hot encoded variables wouldn’t be necessary as this step will definitely eliminate one of them.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_numeric = X._get_numeric_data() #drop non-numeric cols
#Create a copy of X only for the sole purpose of feature selection
X_copy = X.copy()
X_copy_numeric_scaled = pd.DataFrame(scaler.fit_transform(X_numeric), columns=X_numeric.columns)
#X_copy_numeric_scaled.drop('const',axis=1,inplace = True)
X_copy.update(X_copy_numeric_scaled)

#Convert the 'one-hot-encoded' columns back to float type
X_copy[tocategorify] = X_copy[tocategorify].astype('float')
from sklearn.linear_model import LassoCV
reg = LassoCV(cv=10)
reg.fit(X_copy, y)

coef = pd.Series(reg.coef_, index = X_copy.columns).to_frame()
chosen_vars = coef[coef[0]!=0].index.tolist()
print("Lasso picked " + str(len(chosen_vars)) + 
      " out of " +  str(len(coef)) + " variables")

In [ ]:
X_copy.head()

In [ ]:
imp_coef = coef.sort_values(by = 0)
imp_coef = pd.DataFrame(imp_coef[imp_coef[0] != 0])
print("Lasso selected the following Features:")
list(imp_coef.index)

##### Visualizing

In [ ]:
#Temporarily injecting 'Sale Price' column to see the top n correlated with sale price
X_copy['SalePrice'] = HousingPrices_train['SalePrice']
corrmat=X_copy.corr()
n=98
topn_cols=corrmat.nlargest(n,'SalePrice')['SalePrice'].index
corrmat_top15=X_copy[topn_cols].corr()
fig1,ax1=plt.subplots(figsize=(40,40))
sns.heatmap(corrmat_top15,vmax=.8, square=True,ax=ax1,annot=True, fmt='.2f', annot_kws={'size': 12})
X_copy.drop('SalePrice', axis = 1, inplace = True)
top_cols = topn_cols.to_list()

##### Finalizing X, and scaling train & test separately

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
chosen_features = list(set(chosen_vars).intersection(top_cols))
X = X[chosen_features]
C = list(set(tocategorify).intersection(chosen_features))
X[C] = X[C].astype('category') #cate
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=823)
X_train_numeric = X_train._get_numeric_data() #drop non-numeric cols
X_test_numeric = X_test._get_numeric_data() #drop non-numeric cols
scaler = StandardScaler()
X_train_numeric_scaled = pd.DataFrame(scaler.fit_transform(X_train_numeric), 
                                      index=X_train.index,
                                      columns=X_train_numeric.columns)
X_test_numeric_scaled = pd.DataFrame(scaler.transform(X_test_numeric), 
                                     index = X_test.index, 
                                     columns=X_test_numeric.columns)
X_train.shape
X_train.update(X_train_numeric_scaled)
X_test.update(X_test_numeric_scaled)
X_train[C] = X_train[C].astype('float')
X_test[C] = X_test[C].astype('float')
display(X_train.head())
display(X_test.head()) 
X[C] = X[C].astype('float')

Y = HousingPrices_test[chosen_features]
Y[C] = Y[C].astype('category')
Y_numeric = Y._get_numeric_data() #drop non-numeric cols
Y_numeric_scaled = pd.DataFrame(scaler.transform(Y_numeric), 
                                index = Y.index, 
                                columns = Y_numeric.columns)
Y.update(Y_numeric_scaled)
Y[C] = Y[C].astype('float')

## Random Forest

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
k = 10
rf = RandomForestRegressor()
m = [8]
e = [2000]
parameters = {'max_depth': m, 'n_estimators' : e}
rf_reg = GridSearchCV(rf, parameters, 
                      scoring = 'neg_mean_squared_error',
                      cv = k)
rf_reg.fit(X_train,y_train)
print("Best Parameter Values: ")
pd.DataFrame.from_dict(rf_reg.best_params_,orient='index',columns=['Values'])

In [ ]:
best_rfr_model = rf_reg.best_estimator_
predictions = best_rfr_model.predict(X_test)
predictions2 = best_rfr_model.predict(X_train)
d1 = {'Test Data' : [metrics.mean_absolute_error(y_test, predictions),
                metrics.mean_squared_error(y_test, predictions),
                np.sqrt(metrics.mean_squared_error(y_test, predictions))],
     #'Train' : [metrics.mean_absolute_error(y_train, predictions2),
     #           metrics.mean_squared_error(y_train, predictions2),
     #          np.sqrt(metrics.mean_squared_error(y_train, predictions2))]
     }
m = pd.DataFrame(d1,['MAE','MSE','RMSE'])
m.style.format("{:.2f}")

##### Scatter plot of actual values vs predicted values (HousingPrices_train)

In [ ]:
plt.rcParams['figure.figsize'] = (7.0, 5.0)
plt.title("Actual vs Predicted Values")
plt.scatter(y_test,predictions)

##### Residual Histogram – to visualize error distribution

In [ ]:
plt.rcParams['figure.figsize'] = (7.0, 5.0)
plt.title("Actual - Predicted Distribution Plot")
sns.distplot((y_test-predictions),bins=50);

## Gradient Boosting

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

gb = GradientBoostingRegressor()
parameters = {'n_estimators':[3000], 
              'learning_rate':[0.05],
              'max_depth':[4],
              'min_samples_leaf':[15],
              'min_samples_split':[10]}
gb_reg = GridSearchCV(gb, parameters, 
                      scoring = 'neg_mean_squared_error',
                      cv = k)
gb_reg.fit(X_train,y_train)
print("Best Parameter Values: ")
pd.DataFrame.from_dict(gb_reg.best_params_,orient='index',columns=['Values'])

In [ ]:
best_gbr_model = gb_reg.best_estimator_
predictions = best_gbr_model.predict(X_test)
predictions2 = best_gbr_model.predict(X_train)
d1 = {'Test Data' : [metrics.mean_absolute_error(y_test, predictions),
                metrics.mean_squared_error(y_test, predictions),
                np.sqrt(metrics.mean_squared_error(y_test, predictions))],
     #'Train' : [metrics.mean_absolute_error(y_train, predictions2),
     #           metrics.mean_squared_error(y_train, predictions2),
     #          np.sqrt(metrics.mean_squared_error(y_train, predictions2))]
     }
m = pd.DataFrame(d1,['MAE','MSE','RMSE'])
#m['% Difference'] = ((m['Train'] - m['Test']) / (m['Train']) * 100)
m.style.format("{:.2f}")

##### Scatter plot of actual values vs predicted values (HousingPrices_train)

In [ ]:
plt.rcParams['figure.figsize'] = (7.0, 5.0)
plt.title("Actual vs Predicted Values")
plt.scatter(y_test,predictions)

##### Residual Histogram – to visualize error distribution

In [ ]:
plt.rcParams['figure.figsize'] = (7.0, 5.0)
plt.title("Actual - Predicted Distribution Plot")
sns.distplot((y_test-predictions),bins=50);

## Extreme Gradient Boosting

In [ ]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

xgb = xgb.XGBRegressor()
parameters={'n_estimators':[20000], 
            'learning_rate': [0.0015],
            'max_depth':[5]}
xgb_reg = GridSearchCV(xgb, parameters, 
                      scoring = 'neg_mean_squared_error',
                      cv = k)
xgb_reg.fit(X_train,y_train) 
print("Best Parameter Values: ")
pd.DataFrame.from_dict(xgb_reg.best_params_,orient='index',columns=['Values'])

In [ ]:
best_xgbr_model = xgb_reg.best_estimator_
predictions = best_xgbr_model.predict(X_test)
predictions2 = best_xgbr_model.predict(X_train)
d1 = {'Test Data' : [metrics.mean_absolute_error(y_test, predictions),
                metrics.mean_squared_error(y_test, predictions),
                np.sqrt(metrics.mean_squared_error(y_test, predictions))],
     #'Train' : [metrics.mean_absolute_error(y_train, predictions2),
     #           metrics.mean_squared_error(y_train, predictions2),
     #          np.sqrt(metrics.mean_squared_error(y_train, predictions2))]
     }
m = pd.DataFrame(d1,['MAE','MSE','RMSE'])
m.style.format("{:.2f}")

##### Scatter plot of actual values vs predicted values (HousingPrices_train)

In [ ]:
plt.rcParams['figure.figsize'] = (7.0, 5.0)
plt.title("Actual vs Predicted Values")
plt.scatter(y_test,predictions)

##### Residual Histogram – to visualize error distribution

In [ ]:
plt.rcParams['figure.figsize'] = (7.0, 5.0)
plt.title("Actual - Predicted Distribution Plot")
sns.distplot((y_test-predictions),bins=50);

In [ ]:
SalePrice = pd.Series(best_xgbr_model.predict(Y),name='SalePrice')
my_solution = pd.concat([HousingPrices_test['Id'],SalePrice], axis=1)
my_solution.to_csv('my_output_xgbr.csv',index=False)

##### Maybe dealing with outliers can lead to achieving better scores. 
##### Loads of thanks to [Serigne](https://www.kaggle.com/serigne) for sharing the Box-Cox Transformation. 